In [1]:
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Adding AWS S3 Minio configs
sparkConf = (
    SparkConf()
    .set("spark.jars.ivy","/home/brijeshdhaker/.ivy2")
    .set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.0.0")
    .set("spark.executor.heartbeatInterval", "300000")
    .set("spark.network.timeout", "400000")
    .set("spark.hadoop.fs.s3a.endpoint", "http://minio.sandbox.net:9010")
    .set("spark.hadoop.fs.s3a.access.key", "pgm2H2bR7a5kMc5XCYdO")
    .set("spark.hadoop.fs.s3a.secret.key", "zjd8T0hXFGtfemVQ6AH3yBAPASJNXNbVSx5iddqG")
    .set("spark.hadoop.fs.s3a.path.style.access", "true")
    .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    #.set("spark.eventLog.enabled", "true")
    #.set("spark.eventLog.dir", "file:///apps/var/logs/spark-events")
)

spark = (
    SparkSession.builder.master("local[*]").
        appName('spark-sql-notebook').
        config(conf=sparkConf).
        getOrCreate()
)

spark.sparkContext.setLogLevel('ERROR')
spark

25/09/30 15:39:12 WARN Utils: Your hostname, vmware-ubuntu-24.04 resolves to a loopback address: 127.0.1.1; using 192.168.154.133 instead (on interface ens33)
25/09/30 15:39:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/conda/envs/env_python3_11_13/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/brijeshdhaker/.ivy2/cache
The jars for the packages stored in: /home/brijeshdhaker/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c4d6c647-d4e4-4307-a0e3-afed99c94e08;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.0.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.199 in central
:: resolution report :: resolve 208ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.199 from central in [default]
	org.apache.hadoop#hadoop-aws;3.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	------------------

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "8")
print(spark.conf.get("spark.sql.shuffle.partitions"))

In [ ]:
dfFromCSV = spark.read \
    .option("header", True) \
    .option("delimiter", ',') \
    .options(inferSchema='True', delimiter=',') \
    .csv("file:///apps/sandbox/defaultfs/employee.csv")
dfFromCSV.printSchema()
dfFromCSV.show(truncate=False)

In [ ]:
dfFromJSON = spark.read.json("file:///apps/sandbox/defaultfs/people.json")
dfFromJSON.printSchema()
dfFromJSON.show(truncate=False)

In [35]:
employee_df = spark.read.csv("file:///apps/sandbox/defaultfs/employee.csv",
    header=True,
    nullValue="NA",
    inferSchema=True
)

print(employee_df.rdd.getNumPartitions())

1


In [38]:
#
employee_df = employee_df.repartition(2)
print(employee_df.rdd.getNumPartitions())
#
employee_df.show(truncate=False)

2
+------+--------+---------+-----------+------------+----------+--------+--------+
|emp_id|emp_name|emp_role |emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|
+------+--------+---------+-----------+------------+----------+--------+--------+
|7521  |WARD    |SALESMAN |7698       |1982-06-23  |1250      |500     |30      |
|7788  |SCOTT   |ANALYST  |7566       |1984-12-17  |3000      |NULL    |20      |
|7654  |MARTIN  |SALESMAN |7698       |1981-07-21  |1250      |1400    |30      |
|7935  |ROBERT  |ANALYST  |7566       |1984-03-31  |1940      |225     |10      |
|7839  |KING    |PRESIDENT|NULL       |1981-11-17  |5000      |NULL    |10      |
|7566  |JONES   |MANAGER  |7839       |1981-02-04  |2975      |345     |20      |
|7844  |TURNER  |SALESMAN |7698       |1982-09-08  |1500      |0       |30      |
|7876  |ADAMS   |CLERK    |7788       |1980-11-14  |1100      |100     |20      |
|7782  |CLARK   |MANAGER  |7839       |1983-09-06  |2450      |NULL    |10      |
|7900  |JAMES 

## Process Dataframe

### Map Dataframe Partitions

In [ ]:

def applyMap(row):
    bonus = 0
    salary = 0

    if row.emp_comm is None :
        bonus = 0
    else :
        bonus = row.emp_comm


    if row.emp_role == 'ANALYST':
        salary = row.emp_salary + 1000
        bonus = bonus + 100
    elif row.emp_role == 'CLERK':
        salary = row.emp_salary + 1500
        bonus = bonus + 150
    elif row.emp_role == 'MANAGER':
        salary = row.emp_salary + 2000
        bonus = bonus + 200
    elif row.emp_role == 'SALESMAN':
        salary = row.emp_salary + 2500
        bonus = bonus + 250
    else:
        salary = row.emp_salary
        bonus = 0
    
    return (row.emp_id, row.emp_name, row.emp_role, row.emp_manager, row.emp_hiredate, salary, bonus, row.emp_dept)


df1_columns = ["emp_id","emp_name", "emp_role", "emp_manager", "emp_hiredate", "emp_salary", "emp_comm", "emp_dept"]

##
#df1 = employee_df.rdd.map(lambda r: (r.emp_id, r.emp_name, r.emp_role, r.emp_manager, r.emp_hiredate, r.emp_salary, 225, r.emp_dept)).toDF(df1_columns)

##
df1 = employee_df.rdd.map(applyMap).toDF(df1_columns)
df1.show(truncate=False)


+------+--------+---------+-----------+------------+----------+--------+--------+
|emp_id|emp_name|emp_role |emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|
+------+--------+---------+-----------+------------+----------+--------+--------+
|7521  |WARD    |SALESMAN |7698       |1982-06-23  |3750      |750     |30      |
|7788  |SCOTT   |ANALYST  |7566       |1984-12-17  |4000      |100     |20      |
|7654  |MARTIN  |SALESMAN |7698       |1981-07-21  |3750      |1650    |30      |
|7935  |ROBERT  |ANALYST  |7566       |1984-03-31  |2940      |325     |10      |
|7839  |KING    |PRESIDENT|NULL       |1981-11-17  |5000      |0       |10      |
|7566  |JONES   |MANAGER  |7839       |1981-02-04  |4975      |545     |20      |
|7844  |TURNER  |SALESMAN |7698       |1982-09-08  |4000      |250     |30      |
|7876  |ADAMS   |CLERK    |7788       |1980-11-14  |2600      |250     |20      |
|7782  |CLARK   |MANAGER  |7839       |1983-09-06  |4450      |200     |10      |
|7900  |JAMES   

### flatMap Dataframe Partitions

### Process Dataframe Partitions
Unfortunately, PySpark DataFame doesn’t have flatMap() transformation however, DataFrame has explode() SQL function that is used to flatten the column. Below is a complete example.

In [51]:
arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})]
df = spark.createDataFrame(data=arrayData, schema = ['name','knownLanguages','properties'])

from pyspark.sql.functions import explode
df2 = df.select(df.name,explode(df.knownLanguages))
df2.printSchema()
df2.show()

root
 |-- name: string (nullable = true)
 |-- col: string (nullable = true)

+---------+------+
|     name|   col|
+---------+------+
|    James|  Java|
|    James| Scala|
|  Michael| Spark|
|  Michael|  Java|
|  Michael|  NULL|
|   Robert|CSharp|
|   Robert|      |
|Jefferson|     1|
|Jefferson|     2|
+---------+------+



In [53]:
# Sample DataFrame
data = [("Spark is great",), ("Map and FlatMap are useful",)]
df = spark.createDataFrame(data, ["sentence"])

# Use 'explode' function to achieve flatMap-like behavior
df_words = df.withColumn("words", explode(split(col("sentence"), " ")))

df_words.show()

+--------------------+-------+
|            sentence|  words|
+--------------------+-------+
|      Spark is great|  Spark|
|      Spark is great|     is|
|      Spark is great|  great|
|Map and FlatMap a...|    Map|
|Map and FlatMap a...|    and|
|Map and FlatMap a...|FlatMap|
|Map and FlatMap a...|    are|
|Map and FlatMap a...| useful|
+--------------------+-------+



In [39]:
# Using mapPartitions with yield
def formatWithYield(partition_data):
    for record in partition_data:
        role = 'ANALYST' if  record.emp_role == 'CLERK' else 'MANAGER'
        salary = record.emp_salary
        if record.emp_salary <= 1000 :
            salary = record.emp_salary * 10
        
        bonus = record.emp_comm 
        if record.emp_comm is None :
            bonus = 50
        yield (record.emp_id, record.emp_name, record.emp_role, record.emp_manager, record.emp_hiredate, salary, bonus, record.emp_dept)




df2_columns = ["emp_id","emp_name", "emp_role", "emp_manager", "emp_hiredate", "emp_salary", "emp_comm", "emp_dept"]
df2 = employee_df.rdd.mapPartitions(formatWithYield).toDF(df2_columns)
df2.show(truncate=False)



+------+--------+---------+-----------+------------+----------+--------+--------+
|emp_id|emp_name|emp_role |emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|
+------+--------+---------+-----------+------------+----------+--------+--------+
|7521  |WARD    |SALESMAN |7698       |1982-06-23  |1250      |500     |30      |
|7788  |SCOTT   |ANALYST  |7566       |1984-12-17  |3000      |50      |20      |
|7654  |MARTIN  |SALESMAN |7698       |1981-07-21  |1250      |1400    |30      |
|7935  |ROBERT  |ANALYST  |7566       |1984-03-31  |1940      |225     |10      |
|7839  |KING    |PRESIDENT|NULL       |1981-11-17  |5000      |50      |10      |
|7566  |JONES   |MANAGER  |7839       |1981-02-04  |2975      |345     |20      |
|7844  |TURNER  |SALESMAN |7698       |1982-09-08  |1500      |0       |30      |
|7876  |ADAMS   |CLERK    |7788       |1980-11-14  |1100      |100     |20      |
|7782  |CLARK   |MANAGER  |7839       |1983-09-06  |2450      |50      |10      |
|7900  |JAMES   

In [40]:
# Using mapPartitions with iterator
def formatWithIter(partition_data):
    p_data = []
    for record in partition_data:
        role = 'ANALYST' if  record.emp_role == 'CLERK' else 'MANAGER'
        salary = record.emp_salary
        if record.emp_salary <= 1000 :
            salary = record.emp_salary * 10
        
        bonus = record.emp_comm 
        if record.emp_comm is None:
            bonus = 50
        
        p_data.append([record.emp_id, record.emp_name, record.emp_role, record.emp_manager, record.emp_hiredate, salary, bonus, record.emp_dept])
    return iter(p_data)

df3_columns = ["emp_id","emp_name", "emp_role", "emp_manager", "emp_hiredate", "emp_salary", "emp_comm", "emp_dept"]
df3 = employee_df.rdd.mapPartitions(formatWithIter).toDF(df3_columns)
df3.show(truncate=False)

+------+--------+---------+-----------+------------+----------+--------+--------+
|emp_id|emp_name|emp_role |emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|
+------+--------+---------+-----------+------------+----------+--------+--------+
|7521  |WARD    |SALESMAN |7698       |1982-06-23  |1250      |500     |30      |
|7788  |SCOTT   |ANALYST  |7566       |1984-12-17  |3000      |50      |20      |
|7654  |MARTIN  |SALESMAN |7698       |1981-07-21  |1250      |1400    |30      |
|7935  |ROBERT  |ANALYST  |7566       |1984-03-31  |1940      |225     |10      |
|7839  |KING    |PRESIDENT|NULL       |1981-11-17  |5000      |50      |10      |
|7566  |JONES   |MANAGER  |7839       |1981-02-04  |2975      |345     |20      |
|7844  |TURNER  |SALESMAN |7698       |1982-09-08  |1500      |0       |30      |
|7876  |ADAMS   |CLERK    |7788       |1980-11-14  |1100      |100     |20      |
|7782  |CLARK   |MANAGER  |7839       |1983-09-06  |2450      |50      |10      |
|7900  |JAMES   

### Process Dataframe with mapPartitionsWithIndex

In [42]:
#
#
#
def formatWithMapPartitionsWithIndex(partitionIndex, iterator):
    for paartion_data in iterator:
        role = 'ANALYST' if  paartion_data.emp_role == 'CLERK' else 'MANAGER'
        salary = paartion_data.emp_salary
        if paartion_data.emp_salary <= 1000 :
            salary = paartion_data.emp_salary * 10
        
        bonus = paartion_data.emp_comm 
        if paartion_data.emp_comm is None:
            bonus = 50
        
        yield (partitionIndex+1, paartion_data.emp_id, paartion_data.emp_name, role, paartion_data.emp_manager, paartion_data.emp_hiredate, salary, bonus, paartion_data.emp_dept)
    
    #yield (partitionIndex, len(list(iterator)))
    

df4_columns = ["partition","emp_id","emp_name", "emp_role", "emp_manager", "emp_hiredate", "emp_salary", "emp_comm", "emp_dept"]
df4 = employee_df.rdd.mapPartitionsWithIndex(formatWithMapPartitionsWithIndex).toDF(df4_columns)
df4.show(truncate=False)

+---------+------+--------+--------+-----------+------------+----------+--------+--------+
|partition|emp_id|emp_name|emp_role|emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|
+---------+------+--------+--------+-----------+------------+----------+--------+--------+
|1        |7521  |WARD    |MANAGER |7698       |1982-06-23  |1250      |500     |30      |
|1        |7788  |SCOTT   |MANAGER |7566       |1984-12-17  |3000      |50      |20      |
|1        |7654  |MARTIN  |MANAGER |7698       |1981-07-21  |1250      |1400    |30      |
|1        |7935  |ROBERT  |MANAGER |7566       |1984-03-31  |1940      |225     |10      |
|1        |7839  |KING    |MANAGER |NULL       |1981-11-17  |5000      |50      |10      |
|1        |7566  |JONES   |MANAGER |7839       |1981-02-04  |2975      |345     |20      |
|1        |7844  |TURNER  |MANAGER |7698       |1982-09-08  |1500      |0       |30      |
|2        |7876  |ADAMS   |ANALYST |7788       |1980-11-14  |1100      |100     |20      |

In [23]:
employee_df = employee_df.filter(col("emp_salary") > 2000)\
    .select("emp_id", "emp_name", "emp_dept", "emp_salary")\
    .groupby("emp_dept")\
    .count()

In [24]:
employee_df.collect()

[Row(emp_dept=20, count=3),
 Row(emp_dept=10, count=2),
 Row(emp_dept=30, count=1)]

In [25]:
df = spark.range(1,6)
df.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+



In [26]:
df.select("id").show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+



In [27]:
df.select((df.id).alias("##"), col("id").alias("#ID") ,(df.id + 10).alias('###COL')).show()

+---+---+------+
| ##|#ID|###COL|
+---+---+------+
|  1|  1|    11|
|  2|  2|    12|
|  3|  3|    13|
|  4|  4|    14|
|  5|  5|    15|
+---+---+------+



In [28]:
df.selectExpr("id * 5" , "id").show()

+--------+---+
|(id * 5)| id|
+--------+---+
|       5|  1|
|      10|  2|
|      15|  3|
|      20|  4|
|      25|  5|
+--------+---+



In [29]:
names = spark.createDataFrame(data=[(1000, 'Nick'), (1001, 'John'), (1002, 'Frank')], schema=['id', 'name'])
names.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)



In [30]:
names.select('id', 'name').show()

+----+-----+
|  id| name|
+----+-----+
|1000| Nick|
|1001| John|
|1002|Frank|
+----+-----+



#### Q-001. If this value is set to a number other than 200, the number of tasks in the shuffle stage will reflect that new value.

`Adaptive Query Execution (AQE)`: In Spark 3.0 and later, Adaptive Query Execution (AQE) is an optimization that can dynamically adjust the number of shuffle partitions during runtime. AQE can coalesce small shuffle partitions into larger ones, effectively reducing the number of tasks if the data distribution allows for it. If AQE is enabled and determines that fewer partitions are optimal, you will see fewer than 200 tasks.

`Data Volume and Distribution`: If the amount of data being processed is very small, or if the data is highly skewed (meaning a few keys have a disproportionately large amount of data), Spark might not utilize all 200 partitions efficiently, or AQE might optimize the partition count.

`Coalesce or Repartition before GroupBy`: If a coalesce or repartition operation was performed on the DataFrame immediately before the groupBy, it could explicitly set the number of partitions, overriding the default shuffle partition setting for that specific operation.

`Specific Optimization Strategies`: Certain optimization strategies or custom partitioning schemes might be in place that influence the number of partitions used during the groupBy operation, leading to a task count different from 200.

In [31]:
names.select(col("id")).show()

+----+
|  id|
+----+
|1000|
|1001|
|1002|
+----+



In [32]:
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

# Sum of the first 100 whole numbers
rdd = sc.parallelize(range(100 + 1))
rdd.sum()
# 5050

5050